<a href="https://colab.research.google.com/github/yusrilandrian/TugasPraktikumML_167-269/blob/main/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load Data**

In [1]:
#import library 
import pandas as pd
import numpy as np
import os, cv2, shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm as tq
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, GlobalMaxPooling2D, GlobalAveragePooling2D, AveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import files
from keras.preprocessing import image

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Kaggle/

Mounted at /content/drive
/content/drive/MyDrive/Kaggle


In [5]:
file = files.upload()

Saving kaggle.json to kaggle.json


In [16]:
api_token = {"username":"yusrilandrian","key":"0eae344ffba248b35432685b4a980d48"}
import json
import zipfile
import os
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

In [17]:
%cd /content/

/content


In [20]:
! kaggle datasets download -d mikful/orchids

100% 1.07G/1.07G [00:26<00:00, 35.6MB/s]
100% 1.07G/1.07G [00:26<00:00, 43.4MB/s]


In [21]:
ls

drive/  orchids.zip  sample_data/


In [23]:
# extract data zip
! unzip orchids.zip

Output streaming akan dipotong hingga 5000 baris terakhir.
  inflating: dataset/dataset/Brassia/33.Brassia-1024x439.jpg  
  inflating: dataset/dataset/Brassia/330.brs_06_grande.jpeg  
  inflating: dataset/dataset/Brassia/331.Brassia%20verrcosa%201.jpg  
  inflating: dataset/dataset/Brassia/332.brassia-orchid-heiko-koehrer-wagner.jpg  
  inflating: dataset/dataset/Brassia/333.0796_brassia_orquideas_cultivadas.jpg  
  inflating: dataset/dataset/Brassia/334.3382813_Brassia_longissima.jpg  
  inflating: dataset/dataset/Brassia/335.brassia-orqu%C3%ADdea-flores-colecci%C3%B3n-de-im%C3%A1gen__x75165022.jpg  
  inflating: dataset/dataset/Brassia/336.orqu%C3%ADdea-amarilla-del-brassia-7682301.jpg  
  inflating: dataset/dataset/Brassia/337.e67f6fb44a4000dce7ab8e33daa032c5.jpg  
  inflating: dataset/dataset/Brassia/338.orchideeen-brassia-shelob-tolkien-2-branch.jpg  
  inflating: dataset/dataset/Brassia/339.D2xzmKOWkAAVX6a.jpg  
  inflating: dataset/dataset/Brassia/34.Ada_0020_aurantiaca.jpg  
  

In [24]:
ls

dataset/  drive/  orchids.zip  sample_data/


In [25]:
%cd /content/dataset/

/content/dataset


In [26]:
os.listdir('/content/dataset/')

['Odontoglossum',
 'Dendrobium',
 'Epidendrum',
 'Brassavola',
 'Maxillaria',
 'Miltoniopsis',
 'Masdevallia',
 'Paphiopedilum',
 'Encyclia',
 'Miltonia',
 'Angraecum',
 'Vanilla',
 'Zygopetalum',
 'Oncidium',
 'Cymbidium',
 'Brassia',
 'Cattleya',
 'Lycaste',
 'Vanda',
 'dataset',
 'Phalaenopsis']

In [28]:
from PIL import Image
base_dataset = '/content/dataset/'
class_dir = ['Brassia','Miltonia']
for class_item in class_dir:
  cur_dir = base_dataset+"/"+class_item
  dataset = os.listdir(cur_dir)
  for item in dataset:
    if not item.endswith(".jpg"):
        os.remove(os.path.join(cur_dir, item))

In [29]:
file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(base_dataset):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [30]:
# membuat dataframe dari variabel baru
data = pd.DataFrame({'path':full_path,'file_name':file_name,'tag':tag})
print(data.groupby(['tag']).size())

tag
Angraecum         583
Brassavola       1015
Brassia           600
Cattleya          577
Cymbidium         548
Dendrobium        520
Encyclia          616
Epidendrum        609
Lycaste           581
Masdevallia       651
Maxillaria        410
Miltonia          443
Miltoniopsis      588
Odontoglossum     424
Oncidium          447
Paphiopedilum     624
Phalaenopsis      928
Vanda             515
Vanilla           414
Zygopetalum       556
dtype: int64


In [31]:
x = data['path']
y = data['tag']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=300)

In [33]:
X_test, X_val, y_test, y_val = train_test_split(X_train, y_train, test_size=0.7, random_state=100)

In [34]:
# menyatukan ke dalam masing-masing dataframe
data_tr = pd.DataFrame({'path':X_train, 'tag':y_train, 'set':'training'})

data_ts = pd.DataFrame({'path':X_test, 'tag':y_test, 'set':'test'})

data_val = pd.DataFrame({'path':X_val, 'tag':y_val, 'set':'validation'})

In [35]:
print('train size', len(data_tr))
print('val size', len(data_val))
print('test size', len(data_ts))

train size 8736
val size 6116
test size 2620


In [36]:
# melihat proporsi pada masing masing set apakah sudah ok atau masih ada yang ingin diubah
df_all = data_tr.append([data_val,data_ts]).reset_index(drop=1)\

print('===================================================== \n')
print(df_all.groupby(['set','tag']).size(),'\n')

print('===================================================== \n')

#cek sample datanya
df_all.sample(3)


set         tag          
test        Angraecum        145
            Brassavola       231
            Brassia          126
            Cattleya         135
            Cymbidium        128
            Dendrobium       111
            Encyclia         138
            Epidendrum       130
            Lycaste          128
            Masdevallia      146
            Maxillaria        89
            Miltonia          99
            Miltoniopsis     131
            Odontoglossum    101
            Oncidium          99
            Paphiopedilum    150
            Phalaenopsis     209
            Vanda            113
            Vanilla           82
            Zygopetalum      129
training    Angraecum        431
            Brassavola       764
            Brassia          443
            Cattleya         429
            Cymbidium        421
            Dendrobium       382
            Encyclia         471
            Epidendrum       445
            Lycaste          425
            Masd

,path,tag,set
3590,/content/dataset/dataset/Epidendrum/210.105829...,Epidendrum,training
14288,/content/dataset/Brassia/151.Brassia_arcuigera...,Brassia,validation
11336,/content/dataset/Epidendrum/194.depositphotos_...,Epidendrum,validation


In [37]:
#Memasukkan Data pada setiap folder dari data train dan validation
dataset_path = '/content/model_mammals'

data_all = data_tr.append([data_tr, data_val, data_ts]).reset_index(drop=1)

for index, row in tq(data_all.iterrows()):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

0it [00:00, ?it/s]

In [40]:
os.listdir('/content/drive/MyDrive/kaggle/model_dataset/training')

['Paphiopedilum',
 'Lycaste',
 'Masdevallia',
 'Vanilla',
 'Dendrobium',
 'Oncidium',
 'Phalaenopsis',
 'Brassavola',
 'Brassia',
 'Vanda',
 'Encyclia',
 'Cymbidium',
 'Epidendrum',
 'Zygopetalum',
 'Angraecum',
 'Miltonia',
 'Cattleya',
 'Maxillaria',
 'Odontoglossum',
 'Miltoniopsis']

# **Augmentation**

In [41]:
datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [43]:
input_shape = (250, 250)

train = '/content/drive/MyDrive/kaggle/model_dataset/training'
train_generator = datagen.flow_from_directory(train, 
                                                    target_size=input_shape,
                                                    class_mode ='categorical',
                                                    batch_size = 32,
                                                    color_mode ='rgb',
                                                    shuffle = True) 

validation = '/content/drive/MyDrive/kaggle/model_dataset/validation'
validation_generator = datagen.flow_from_directory(validation,
                                                        target_size=input_shape,
                                                        class_mode ='categorical',
                                                        color_mode ='rgb',
                                                        batch_size = 32,
                                                        shuffle = True)

test = '/content/drive/MyDrive/kaggle/model_dataset/test'
test_generator = datagen.flow_from_directory(test,
                                                  target_size=input_shape,
                                                  batch_size = 32,
                                                  color_mode ='rgb',
                                                  shuffle = False)

Found 5781 images belonging to 20 classes.
Found 2111 images belonging to 20 classes.


FileNotFoundError: ignored